# Connect to ChatGPT RBI Endpoint
In this notebook, you'll manage to connect to RBI Azure ChatGPT endpoint.  
The step by step preparation is described [here](https://wiki.rbinternational.com/confluence/display/AUG/GenAI+API+Gateway+Connection)

In [0]:
!pip install openai
!pip install aa-llm-utils
!pip install aa-llm-tools
!pip install llama-index
!pip install llama-index-embeddings-azure-openai
!pip install llama-index-llms-azure-openai
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-instructor
!pip install langchain
dbutils.library.restartPython()

### Importing essential libraries

In [0]:
import os
import sys
import glob
import json

import openai
from openai import OpenAI
from openai import AzureOpenAI

## OpenAI
from aa_llm_utils.llms import RBIOpenAI
from aa_llm_utils.llms import RBIAsyncOpenAI

## LlamaIndex
from aa_llm_utils.llms import RBILIOpenAI
from aa_llm_utils.embeddings import RBILIOpenAIEmbeddings
# this works for llama_index>=0.8.63:
from aa_llm_utils.embeddings import RBILIAzureOpenAIEmbeddings
from llama_index.embeddings.instructor import InstructorEmbedding

## Langchain
#from aa_llm_utils.llms import RBILCChatOpenAI
#from aa_llm_utils.embeddings import RBILCOpenAIEmbeddings

In [0]:
print(dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get())

#### Client ID taken from the [branch of the ASA Manifest file](https://code.rbi.tech/raiffeisen/dsa-asa-manifest/blob/ASAOperator/ASAOperatorLog/log.md)

In [0]:
client_id = "dc-ze5pntur9ct287shfvx857k1y"

#### Private key retrieved from previously saved keyfile.json file

In [0]:
with open("/Workspace/Users/renato.rocha-souza@rbinternational.com/Keys/keyfile.json", "r") as fh:
        key = json.load(fh)

#### [Connecting directly to OpenAI](https://code.rbi.tech/raiffeisen/aa-llm-utils/blob/main/notebooks/openai.ipynb)

In [0]:
client = RBIOpenAI(key=key, client_id=client_id)
async_client = RBIAsyncOpenAI(key=key, client_id=client_id)

In [0]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that translates English to French."
        },
        {
            "role": "user",
            "content": "Translate this sentence from English to French. I love programming.",
        }
    ],
    model="gpt-35-turbo",
    temperature=0,
    stream=False
)
#chat_completion
print(chat_completion.choices[0].message.content)

In [0]:
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
    ],
    model="gpt-35-turbo",
    temperature=0,
    stream=False
)

print(chat_completion.choices[0].message.content)

#### [Connecting using LlamaIndex](https://code.rbi.tech/raiffeisen/aa-llm-utils/blob/main/notebooks/llama_index.ipynb) framework

In [0]:
llm = RBILIOpenAI(model="gpt-35-turbo-16k", key=key, client_id=client_id)
#llm = RBILIOpenAI(model="gpt-4", key=key, client_id=client_id)
embed_model = RBILIOpenAIEmbeddings(key=key, client_id=client_id)
from llama_index.core.llms import ChatMessage, MessageRole
#from llama_index import ServiceContext

In [0]:
msg = "Joel, Bndy and Lorenz go for a beer to celebrate"
resp = llm.complete(msg)
print(resp.text)

In [0]:
resp = llm.chat(
    [
        ChatMessage(content="You are a helpful assistant that translates English to French.", 
                    role=MessageRole.SYSTEM),
        ChatMessage(content="Translate this sentence from English to French. I love programming.", 
                    role=MessageRole.USER),
    ]
)
print(resp.message)
print(resp.message.content)

In [0]:
embeddings = embed_model.get_text_embedding("It is raining cats and dogs here!")
embeddings[0:10]

In [0]:
embeddings = embed_model.get_text_embedding("It is raining cats and dogs here!")
embeddings

#### [Connecting using Langchain](https://code.rbi.tech/raiffeisen/aa-llm-utils/blob/main/notebooks/langchain.ipynb) framework

In [0]:
chat_llm = RBILCChatOpenAI(key=key, client_id=client_id, model="gpt-35-turbo", temperature=0)
embeddings = RBILCOpenAIEmbeddings(key=key, client_id=client_id,)
from langchain.schema import HumanMessage, SystemMessage

In [0]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
chat_llm.invoke(messages)

In [0]:
text = "This is a test query."
query_result = embeddings.embed_query(text)
query_result